In [1]:
import os
if os.getcwd().endswith("experiments"):
    os.chdir("..")

os.getcwd()

'/home/lars/code/python/context-aware-flow-matching'

In [2]:
from torch.utils.data import random_split
from lightning import seed_everything

from pathlib import Path

from lightning_trainable.utils import find_checkpoint

from src.models import Model
from src.datasets import ModelNet10Dataset

In [3]:
seed_everything(42)

Seed set to 42


42

In [4]:
dataset = ModelNet10Dataset("data/modelnet10")

Found non-empty ModelNet10Dataset in data/modelnet10, skipping download...


Pre-Loading Meshes: 100%|███████████████████████████████████████████████████████████████████| 4899/4899 [00:54<00:00, 90.66it/s]


In [5]:
train_data, val_data, test_data = random_split(dataset, [0.8, 0.1, 0.1])

In [6]:
hparams = dict(
    batch_size=256,
    subset_size=256,
    features=3,
    embeddings=768,
    accumulate_batches=2,
    checkpoint_segments=16,
    gamma=0.5,
    max_epochs=10_000,
    optimizer=dict(
        name="AdamW",
        kwargs=dict(
            weight_decay=0.01,
        ),
    ),
    lr_scheduler=dict(
        name="OneCycleLR",
        kwargs=dict(
            max_lr=1e-4,
            div_factor=1e1,
            final_div_factor=1e6,
        )
    ),
    model_checkpoint=dict(
        monitor="validation/loss",
        save_last=True,
        every_n_epochs=10,
        save_top_k=10,
    ),
    optimal_transport=dict(
        epsilon=0.05,
        max_steps=25_000,
        atol=3e-5,
    ),
    gradient_clip=1.0,
    num_workers=14,
    accelerator="gpu",
    devices=-1,
)

In [ ]:
# initial training
# model = Model(hparams, train_data, val_data, test_data)
# model.fit(trainer_kwargs=dict(log_every_n_steps=4, check_val_every_n_epoch=4))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | encoder | CheckpointedSequential | 27.0 M
1 | flow    | CheckpointedSequential | 20.7 M
---------------------------------------------------
47.7 M    Trainable params
0         Non-trainable params
47.7 M    Total params
190.975   Total estimated model params size (MB)


Sanity Checking: |                                                                                        | 0/…

Training: |                                                                                               | 0/…

Validation: |                                                                                             | 0/…

/home/lars/code/python/context-aware-flow-matching/src/lightning-trainable/src/lightning_trainable/metrics/sinkhorn.py:72: Sinkhorn-Knopp did not converge (badness: 7.0e-05). Consider relaxing epsilon (5.4e-02) or increasing the number of iterations (10000).


Validation: |                                                                                             | 0/…

In [ ]:
# initial training with weights initialized from a previous run
cp = find_checkpoint(version=9)
model = Model.load_from_checkpoint(cp, hparams=hparams, train_data=train_data, val_data=val_data, test_data=test_data)
model.fit(trainer_kwargs=dict(log_every_n_steps=4, check_val_every_n_epoch=4))

In [ ]:
# resume training
# cp = find_checkpoint(version=14)
# cp = Path(cp)
# hparams = Model.hparams_type.from_yaml(cp.parent.parent / "hparams.yaml")
# model = Model(hparams, train_data, val_data, test_data)
# model.fit(trainer_kwargs=dict(log_every_n_steps=4, check_val_every_n_epoch=4, callbacks=model.configure_callbacks()), fit_kwargs=dict(ckpt_path=cp))